## Error correction code

In [16]:
import numpy as np
from typing import Tuple, List, Optional

class LinearCode:
    """
    A comprehensive implementation of linear error-correcting codes
    focusing on the key matrices: Generator matrix G and Parity-check matrix H
    """
    
    def __init__(self, G: np.ndarray):
        """
        Initialize linear code with generator matrix G
        G should be a k×n matrix in systematic form [I_k | A]
        """
        self.G = G % 2  # Ensure binary
        self.k, self.n = G.shape  # k = dimension, n = code length
        
        # Derive parity-check matrix H from G
        self.H = self._derive_parity_check_matrix()
        
        # Calculate code parameters
        self.d_min = self._calculate_minimum_distance()
        self.t = (self.d_min - 1) // 2  # Error correction capability
        
    def _derive_parity_check_matrix(self) -> np.ndarray:
        """
        Derive parity-check matrix H from generator matrix G
        For systematic G = [I_k | A], we have H = [A^T | I_(n-k)]
        Property: G * H^T = 0
        """
        if self._is_systematic():
            # Extract parity part A from systematic form [I_k | A]
            A = self.G[:, self.k:]
            I_r = np.eye(self.n - self.k, dtype=int)
            H = np.hstack([A.T, I_r])
        else:
            # For non-systematic G, use Gaussian elimination
            H = self._compute_parity_check_general()
        
        return H % 2
    
    def _is_systematic(self) -> bool:
        """Check if generator matrix is in systematic form [I_k | A]"""
        identity_part = self.G[:, :self.k]
        expected_identity = np.eye(self.k, dtype=int)
        return np.array_equal(identity_part, expected_identity)
    
    def _compute_parity_check_general(self) -> np.ndarray:
        """Compute parity-check matrix for general (non-systematic) G"""
        # This is more complex - for simplicity, we'll convert to systematic form first
        G_sys = self._to_systematic_form()
        A = G_sys[:, self.k:]
        I_r = np.eye(self.n - self.k, dtype=int)
        return np.hstack([A.T, I_r]) % 2
    
    def _to_systematic_form(self) -> np.ndarray:
        """Convert generator matrix to systematic form using Gaussian elimination"""
        G_work = self.G.copy()
        
        # Gaussian elimination to get [I_k | A] form
        for i in range(self.k):
            # Find pivot
            pivot_row = i
            for j in range(i + 1, self.k):
                if G_work[j][i] == 1:
                    pivot_row = j
                    break
            
            # Swap rows if needed
            if pivot_row != i and G_work[pivot_row][i] == 1:
                G_work[[i, pivot_row]] = G_work[[pivot_row, i]]
            
            # Make diagonal element 1 and eliminate column
            if G_work[i][i] == 1:
                for j in range(self.k):
                    if j != i and G_work[j][i] == 1:
                        G_work[j] = (G_work[j] + G_work[i]) % 2
        
        return G_work
    
    def _calculate_minimum_distance(self) -> int:
        """Calculate minimum distance of the code"""
        min_weight = float('inf')
        
        # Check all non-zero codewords
        for i in range(1, 2**self.k):  # Skip zero codeword
            # Convert i to binary vector of length k
            info_bits = np.array([int(b) for b in format(i, f'0{self.k}b')])
            codeword = self.encode(info_bits)
            weight = np.sum(codeword)
            min_weight = min(min_weight, weight)
        
        return int(min_weight)
    
    def encode(self, message: np.ndarray) -> np.ndarray:
        """Encode k-bit message to n-bit codeword using G"""
        if len(message) != self.k:
            raise ValueError(f"Message must be {self.k} bits long")
        return (message @ self.G) % 2
    
    def syndrome(self, received: np.ndarray) -> np.ndarray:
        """Compute syndrome s = H * r^T for received word r"""
        if len(received) != self.n:
            raise ValueError(f"Received word must be {self.n} bits long")
        return (self.H @ received.T) % 2
    
    def is_codeword(self, word: np.ndarray) -> bool:
        """Check if a word is a valid codeword (syndrome = 0)"""
        s = self.syndrome(word)
        return np.all(s == 0)
    
    def detect_errors(self, received: np.ndarray) -> Tuple[bool, np.ndarray]:
        """Detect if errors occurred and return syndrome"""
        s = self.syndrome(received)
        has_errors = not np.all(s == 0)
        return has_errors, s
    
    def correct_single_error(self, received: np.ndarray) -> Tuple[np.ndarray, bool]:
        """Correct single error using syndrome lookup"""
        s = self.syndrome(received)
        
        # If syndrome is zero, no errors
        if np.all(s == 0):
            return received, True
        
        # Look for syndrome in columns of H (single error correction)
        corrected = received.copy()
        
        for i in range(self.n):
            if np.array_equal(s, self.H[:, i]):
                # Error in position i
                corrected[i] = (corrected[i] + 1) % 2
                return corrected, True
        
        # Syndrome doesn't match any column - uncorrectable
        return received, False
    
    def get_all_codewords(self) -> List[np.ndarray]:
        """Generate all codewords of the code"""
        codewords = []
        for i in range(2**self.k):
            info_bits = np.array([int(b) for b in format(i, f'0{self.k}b')])
            codeword = self.encode(info_bits)
            codewords.append(codeword)
        return codewords
    
    def print_code_info(self):
        """Print comprehensive information about the code"""
        print(f"Linear Code [{self.n}, {self.k}, {self.d_min}]")
        print(f"Code length n = {self.n}")
        print(f"Dimension k = {self.k}")
        print(f"Minimum distance d = {self.d_min}")
        print(f"Error correction capability t = {self.t}")
        print(f"Error detection capability = {self.d_min - 1}")
        print(f"Code rate R = {self.k}/{self.n} = {self.k/self.n:.3f}")
        print(f"Redundancy = {self.n - self.k} parity bits")
        print()
        
        print("Generator Matrix G:")
        print(self.G)
        print()
        
        print("Parity-Check Matrix H:")
        print(self.H)
        print()
        
        # Verify G * H^T = 0
        product = (self.G @ self.H.T) % 2
        print("Verification G * H^T =")
        print(product)
        print(f"Is zero matrix: {np.all(product == 0)}")
        print()


def demo_hamming_code():
    """Demonstrate Hamming(7,4) code - classic example"""
    print("HAMMING (7,4) CODE DEMONSTRATION")
    print("=" * 50)
    
    # Hamming(7,4) generator matrix in systematic form
    G_hamming = np.array([
        [1, 0, 0, 0, 1, 1, 0],  # Information bits: 1000, parity: 110
        [0, 1, 0, 0, 1, 0, 1],  # Information bits: 0100, parity: 101
        [0, 0, 1, 0, 0, 1, 1],  # Information bits: 0010, parity: 011
        [0, 0, 0, 1, 1, 1, 1]   # Information bits: 0001, parity: 111
    ])
    
    hamming = LinearCode(G_hamming)
    hamming.print_code_info()
    
    # Test encoding
    print("ENCODING EXAMPLES:")
    print("-" * 20)
    test_messages = [
        [1, 0, 1, 0],
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [1, 0, 0, 1]
    ]
    
    for msg in test_messages:
        codeword = hamming.encode(np.array(msg))
        print(f"Message {msg} -> Codeword {codeword}")
    
    print()
    
    # Test error detection and correction
    print("ERROR CORRECTION EXAMPLES:")
    print("-" * 30)
    
    original_msg = np.array([1, 0, 1, 0])
    codeword = hamming.encode(original_msg)
    print(f"Original message: {original_msg}")
    print(f"Transmitted codeword: {codeword}")
    
    # Introduce single error
    received = codeword.copy()
    error_pos = 2
    received[error_pos] = (received[error_pos] + 1) % 2
    print(f"Received (error in pos {error_pos}): {received}")
    
    # Detect and correct
    has_errors, syndrome_val = hamming.detect_errors(received)
    print(f"Errors detected: {has_errors}")
    print(f"Syndrome: {syndrome_val}")
    
    corrected, success = hamming.correct_single_error(received)
    print(f"Corrected codeword: {corrected}")
    print(f"Correction successful: {success}")
    print(f"Matches original: {np.array_equal(codeword, corrected)}")
    print()


def demo_custom_code():
    """Demonstrate a custom linear code"""
    print("CUSTOM LINEAR CODE DEMONSTRATION")
    print("=" * 40)
    
    # Create a [6,3,3] code
    G_custom = np.array([
        [1, 0, 0, 1, 1, 0],
        [0, 1, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 1]
    ])
    
    custom_code = LinearCode(G_custom)
    custom_code.print_code_info()
    
    # Show all codewords
    print("ALL CODEWORDS:")
    print("-" * 15)
    codewords = custom_code.get_all_codewords()
    for i, cw in enumerate(codewords):
        weight = np.sum(cw)
        binary_i = format(i, f'0{custom_code.k}b')
        print(f"{binary_i} -> {cw} (weight {weight})")
    
    print()
    
    # Test syndrome calculation
    print("SYNDROME EXAMPLES:")
    print("-" * 20)
    
    test_words = [
        [1, 0, 0, 1, 1, 0],  # Valid codeword
        [1, 0, 0, 1, 1, 1],  # Single error
        [1, 1, 0, 1, 1, 0],  # Single error
        [0, 0, 0, 1, 1, 1],  # Multiple errors
    ]
    
    for word in test_words:
        word_arr = np.array(word)
        is_valid = custom_code.is_codeword(word_arr)
        syndrome_val = custom_code.syndrome(word_arr)
        print(f"Word {word}: syndrome {syndrome_val}, valid: {is_valid}")


def demonstrate_matrices_relationship():
    """Show the fundamental relationship between G and H matrices"""
    print("MATRIX RELATIONSHIPS")
    print("=" * 25)
    
    # Simple [5,2,3] code for clear demonstration
    G = np.array([
        [1, 0, 1, 1, 0],
        [0, 1, 1, 0, 1]
    ])
    
    code = LinearCode(G)
    
    print("Generator Matrix G (k×n):")
    print(G)
    print()
    
    print("Parity-Check Matrix H ((n-k)×n):")
    print(code.H)
    print()
    
    print("Key Relationships:")
    print("1. G * H^T = 0 (orthogonality)")
    product = (G @ code.H.T) % 2
    print(f"   G * H^T = \n{product}")
    print()
    
    print("2. For systematic G = [I_k | A], we have H = [A^T | I_(n-k)]")
    print(f"   Identity part of G: \n{G[:, :code.k]}")
    print(f"   Parity part A: \n{G[:, code.k:]}")
    print(f"   A^T in H: \n{code.H[:, :code.k]}")
    print(f"   Identity in H: \n{code.H[:, code.k:]}")
    print()
    
    print("3. Syndrome calculation: s = H * r^T")
    test_word = np.array([1, 1, 0, 1, 1])
    syndrome = code.syndrome(test_word)
    print(f"   For received word {test_word}:")
    print(f"   Syndrome = {syndrome}")


if __name__ == "__main__":
    demonstrate_matrices_relationship()
    print("\n" + "="*60 + "\n")
    
    demo_hamming_code()
    print("\n" + "="*60 + "\n")
    
    demo_custom_code()
    

MATRIX RELATIONSHIPS
Generator Matrix G (k×n):
[[1 0 1 1 0]
 [0 1 1 0 1]]

Parity-Check Matrix H ((n-k)×n):
[[1 1 1 0 0]
 [1 0 0 1 0]
 [0 1 0 0 1]]

Key Relationships:
1. G * H^T = 0 (orthogonality)
   G * H^T = 
[[0 0 0]
 [0 0 0]]

2. For systematic G = [I_k | A], we have H = [A^T | I_(n-k)]
   Identity part of G: 
[[1 0]
 [0 1]]
   Parity part A: 
[[1 1 0]
 [1 0 1]]
   A^T in H: 
[[1 1]
 [1 0]
 [0 1]]
   Identity in H: 
[[1 0 0]
 [0 1 0]
 [0 0 1]]

3. Syndrome calculation: s = H * r^T
   For received word [1 1 0 1 1]:
   Syndrome = [0 0 0]


HAMMING (7,4) CODE DEMONSTRATION
Linear Code [7, 4, 3]
Code length n = 7
Dimension k = 4
Minimum distance d = 3
Error correction capability t = 1
Error detection capability = 2
Code rate R = 4/7 = 0.571
Redundancy = 3 parity bits

Generator Matrix G:
[[1 0 0 0 1 1 0]
 [0 1 0 0 1 0 1]
 [0 0 1 0 0 1 1]
 [0 0 0 1 1 1 1]]

Parity-Check Matrix H:
[[1 1 0 1 1 0 0]
 [1 0 1 1 0 1 0]
 [0 1 1 1 0 0 1]]

Verification G * H^T =
[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0